In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np


In [90]:
# 加载数据集
data = pd.read_csv('cleaned_data.csv')

print('数据基本信息：')
data.info()

数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Fe               187 non-null    float64
 1   Co               187 non-null    float64
 2   Mn               187 non-null    float64
 3   Al               187 non-null    float64
 4   Ni               187 non-null    float64
 5   B                187 non-null    float64
 6   Hf               187 non-null    float64
 7   Si               187 non-null    float64
 8   Cu               187 non-null    float64
 9   P                187 non-null    float64
 10  Mo               187 non-null    float64
 11  Cr               187 non-null    float64
 12  La               187 non-null    float64
 13  Ti               187 non-null    float64
 14  Sn               187 non-null    float64
 15  V                187 non-null    float64
 16  Ga               187 non-null    float64
 17  Nb      

In [91]:
# 查看列名
column_names = data.columns
print(column_names)

Index(['Fe', 'Co', 'Mn', 'Al', 'Ni', 'B', 'Hf', 'Si', 'Cu', 'P', 'Mo', 'Cr',
       'La', 'Ti', 'Sn', 'V', 'Ga', 'Nb', 'Zr', 'C', 'Gd', 'Y', 'Ta', 'DO3',
       'BCC', 'FCC', 'HCP', 'Orthorhombic', 'hexagonal', 'Tetragonal',
       'Primitive Cubic', 'Ms', 'Hc', 'Hc_log'],
      dtype='object')


In [92]:
# 假设 data 是已经定义好的 DataFrame
# 提取特征变量 x 和目标变量 y（多目标）
x = data.drop(columns=['Ms', 'Hc', 'Hc_log'])
y = data[['Ms', 'Hc_log']]  # 变成包含两个目标变量的 DataFrame

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 创建 GBDT 回归模型并拟合数据
# 针对多目标问题，为每个目标单独创建一个 GBDT 模型
model_ms = GradientBoostingRegressor(n_estimators=100, random_state=42)
model_hc = GradientBoostingRegressor(n_estimators=100, random_state=42)

model_ms.fit(x_train, y_train['Ms'])
model_hc.fit(x_train, y_train['Hc_log'])

# 在测试集上进行预测
y_pred_ms = model_ms.predict(x_test)
y_pred_hc = model_hc.predict(x_test)
y_pred = pd.DataFrame({'Ms': y_pred_ms, 'Hc_log': y_pred_hc})

In [93]:
# 计算均方误差（MSE）
mse_ms = mean_squared_error(y_test['Ms'], y_pred_ms)
mse_hc = mean_squared_error(y_test['Hc_log'], y_pred_hc)

# 计算决定系数（R²）
r2_ms = r2_score(y_test['Ms'], y_pred_ms)
r2_hc = r2_score(y_test['Hc_log'], y_pred_hc)

print(f"Ms 均方误差 (MSE): {mse_ms}")
print(f"Ms 决定系数 (R²): {r2_ms}")
print(f"Hc 均方误差 (MSE): {mse_hc}")
print(f"Hc 决定系数 (R²): {r2_hc}")

Ms 均方误差 (MSE): 543.972692789205
Ms 决定系数 (R²): 0.8293846365452029
Hc 均方误差 (MSE): 0.8488544369571884
Hc 决定系数 (R²): 0.7471789115451946


In [122]:
# 定义复合指标参数（调整λ以平衡Ms和Hc的重要性）
lambda_param = 100  # 示例值，需根据需求调整

# 初始化当前最佳复合指标
current_composite = (y_train['Ms'] - lambda_param * np.exp(y_train['Hc_log'])).max()
print(f"初始最佳复合指标（Ms - λ*Hc）: {current_composite}")

初始最佳复合指标（Ms - λ*Hc）: -12.63871392649699


In [123]:
# 加载候选搜索空间
test_data = pd.read_csv("test_data_SHAP.csv")
x_search = test_data[x.columns]  # 确保特征列一致


In [124]:
# 对所有数据进行预测
pred_ms = model_ms.predict(x_search)
pred_hc = model_hc.predict(x_search)

# 计算候选样本的复合指标 c_hat = Ms^hat - λ * exp(Hc^hat)
c_hat = (pred_ms) - lambda_param * (pred_hc)

# 计算期望改进 Δc = c_hat - current_composite
delta_c = c_hat - current_composite
ie = np.where(delta_c > 0, delta_c, 0)  # 只保留正的改进值

# 选出 ie 最大的 10 个样本
top10_idx = np.argsort(ie)[-10:]  # 排序后取最后10个索引
top10_ie = ie[top10_idx]

final_top10_df = x_search.iloc[top10_idx].copy()
final_top10_df["pred_Ms"] = pred_ms[top10_idx]
final_top10_df["pred_Hc_log"] = pred_hc[top10_idx]
final_top10_df["c_hat"] = c_hat[top10_idx]
final_top10_df["ie"] = top10_ie

In [125]:
# 最终输出 ie 最大的 10 个样本并保存到 Excel 文件
print("\n========== 最终 ie 最大的 10 个样本 ==========")
if 'final_top10_df' in locals():
    print(final_top10_df)
    # 将结果保存到 Excel 文件（文件名可以根据需要调整）
    final_top10_df.to_excel("final_top10_SHAP.xlsx", index=False)
    print("最终的 10 个样本已保存到 'final_top10.xlsx' 文件中。")
else:
    print("未获取到 ie 最大的 10 个样本信息。")


========== 最终 ie 最大的 10 个样本 ==========
         Fe   Co   Mn    Al   Ni     B   Hf    Si   Cu    P  ...  FCC  HCP  \
87874  62.0  0.0  0.8   4.1  0.0  13.1  0.0  17.5  0.0  0.0  ...  1.0  1.0   
83723  64.9  0.0  3.8   5.1  0.0  12.9  0.0   5.7  0.0  0.0  ...  1.0  0.0   
30440  60.3  0.0  9.7   4.8  0.0  12.9  0.0  12.3  0.0  0.0  ...  1.0  0.0   
694    64.3  0.0  4.1   6.6  0.0  13.2  0.0   8.6  0.0  0.0  ...  1.0  0.0   
82784  37.4  0.0  6.9  14.4  0.0  13.7  0.0  15.2  0.0  0.0  ...  0.0  1.0   
60658  55.4  0.0  3.2  11.1  0.0  13.5  0.0  13.1  0.0  0.0  ...  0.0  0.0   
39858  52.4  0.0  4.2   7.1  0.0  13.9  0.0  15.4  0.0  0.0  ...  1.0  0.0   
90640  58.8  0.0  8.3   0.7  0.0  13.0  0.0  12.4  0.0  0.0  ...  1.0  1.0   
28740  56.8  0.0  7.2   4.5  0.0  14.4  0.0  12.9  0.0  0.0  ...  1.0  1.0   
63164  53.9  0.0  6.8   9.0  0.0  13.7  0.0  14.3  0.0  0.0  ...  1.0  0.0   

       Orthorhombic  hexagonal  Tetragonal  Primitive Cubic     pred_Ms  \
87874           0.0       

In [98]:
# # ============= 8. 最终输出 ie 最大的 6 个样本并保存到 Excel 文件 =============
# print("\n========== 最终 ie 最大的 6 个样本 ==========")
# if 'final_top6_df' in locals():
#     print(final_top6_df)
#     # 将结果保存到 Excel 文件（文件名可以根据需要调整）
#     final_top6_df.to_excel("final_top6.xlsx", index=False)
#     print("最终的 6 个样本已保存到 'final_top6.xlsx' 文件中。")
# else:
#     print("未获取到 ie 最大的 6 个样本信息。")